In [20]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

Downloading Data from the link

In [21]:
src = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
content = requests.get(src).text
soup = BeautifulSoup(content,'lxml')


In [22]:
table = soup.find('table',{'class':'wikitable sortable'})

Creating a DataFrame

In [23]:
columnnam = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(columns = columnnam)
df

,PostalCode,Borough,Neighborhood


In [31]:
dfcontent = soup.find('div', class_='mw-parser-output')
table = dfcontent.table.tbody

postcode = 0
borough = 0
neighborhood = 0
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    df = df.append({'PostalCode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
    
#Cleaning Data

df = df[df.Borough!='Not assigned']
df = df[df.Borough!= 0]
df.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1
                                 
df2 = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print(df2.shape)
df2.head()


(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae"


Data Cleaning

In [34]:
df2 = df2.dropna()
empty = 'Not assigned'
df2 = df2[(df2.Postalcode != empty ) & (df2.Borough != empty) & (df2.Neighborhood != empty)

SyntaxError: unexpected EOF while parsing (<ipython-input-34-b9ff52ee2925>, line 3)

In [35]:
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae"


In [37]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df2.groupby(['PostalCode', 'Borough'])
df3 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [41]:
print('The DataFrame shape is', df3.shape)
df3.head(10)


The DataFrame shape is (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae"
5,M1J,Scarborough,"Scarborough Village, Scarborough Village, Scar..."
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park, E..."
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge, Clairlea, Gol..."
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village Wes..."
9,M1N,Scarborough,"Birch Cliff, Cliffside West, Birch Cliff, Clif..."
